In [206]:
import pandas as pd 

from sklearn.metrics import f1_score, confusion_matrix
import numpy as np

## Conjunto de dados

In [207]:
df = pd.read_csv("example.csv")

y_true, y_pred = df["True"], df["Predicted"]

res = f1_score(y_true, y_pred, average="macro")

print("F-score: ", round(res, 3))

F-score:  0.733


In [208]:
def check_word_in_sentence(word, sentence):
    # Dividindo a frase
    words_in_sentence = sentence.split()
    
    # Verificando se a palava está na frase 
    if word in words_in_sentence:
        return True
    else:
        return False

In [209]:
# Adicionando uma coluna para cada termo avaliado 

df['women'] = [check_word_in_sentence('women', t) for t in df['Example'].values]
df['men'] = [check_word_in_sentence('men', t) for t in df['Example'].values]


df


,Index,Example,True,Predicted,women,men
0,1,filthy men,1,1,False,True
1,2,filthy women,1,1,True,False
2,3,we should not let men suffer,0,0,False,True
3,4,we should not let women suffer,0,1,True,False
4,5,men should be loved,0,0,False,True
5,6,women should be loved,0,1,True,False
6,7,they are good men,0,0,False,True
7,8,they are good women,0,1,True,False
8,9,I hate all men,1,1,False,True
9,10,I hate all women,1,1,True,False


## Métricas de viés

In [210]:
def bias_metrics(df, y_true, y_pred, terms = ['women', 'men']):
    # calculando o FPR e FNR    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    #False positive rate
    fpr = fp/(fp+tn)
       
    print("fpr:", round(fpr,3 ))
   
    #False negative rate
    fnr = fn/(fn+tp)
    
    fped = 0
    fned = 0
    
    for t in terms:
        df_aux = df[df[t]]
        
        y_true_t, y_pred_t = df_aux["True"], df_aux["Predicted"]
        tn, fp, fn, tp = confusion_matrix(y_true_t, y_pred_t).ravel()
    
        #False positive rate
        fpr_t = fp/(fp+tn)
        

        #False negative rate
        fnr_t = fn/(fn+tp)
        
        
        fped += np.abs(fpr-fpr_t)
        
        fned += np.abs(fnr-fnr_t)
                
    return (round(fped,3), round(fned,3))      
    

In [211]:
fped, fned = bias_metrics(df, y_true, y_pred, terms = ['women', 'men'])

print(fped, fned)

fpr: 0.5
1.0 0.0


# Exemplo sem viés de gênero

In [212]:
df_2 = pd.read_csv("example_2.csv")

y_true, y_pred = df_2["True"], df_2["Predicted"]

res = f1_score(y_true, y_pred, average="macro")

print("F-score: ", round(res, 3))

F-score:  0.583


In [213]:
# Adicionando informações sobre qual coluna possui um termo de identidade
df_2['women'] = [check_word_in_sentence('women', t) for t in df_2['Example'].values]
df_2['men'] = [check_word_in_sentence('men', t) for t in df_2['Example'].values]
df_2 

,Index,Example,True,Predicted,women,men
0,1,men,1,1,False,True
1,2,women,1,1,True,False
2,3,men,0,0,False,True
3,4,women,0,0,True,False
4,5,men,0,1,False,True
5,6,women,0,1,True,False
6,7,men,0,1,False,True
7,8,women,0,1,True,False
8,9,men,1,1,False,True
9,10,women,1,1,True,False


In [214]:
fped, fned = bias_metrics(df_2, y_true, y_pred, terms = ['women', 'men'])

print("FPED: ", fped)

print("FNED: ", fned)

fpr: 0.667
FPED:  0.0
FNED:  0.0
